## Retrieving Data

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Calgary"

req = requests.get(WIKI_URL)
soup = BeautifulSoup(req.content, 'lxml')
table_classes = {"class": ["sortable", "plainrowheaders"]}
wikitables = soup.findAll("table", table_classes)
right_table=soup.find('table', class_='wikitable sortable')
#print(right_table)
A=[]
B=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==12:
        A.append(cells[0].find(text=True))
        B.append(cells[4].find(text=True))
        
        
df=pd.DataFrame(A,columns=['Neighborhood'])
df['Type']=B
df.head()

,Neighborhood,Type
0,Abbeydale,Residential
1,Acadia,Residential
2,Albert Park/Radisson Heights,Residential
3,Altadore,Residential
4,Alyth/Bonnybrook,Industrial


## Get Latitude and Longitude

In [2]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='fa')
list_latitude=[]
list_longitude=[]
city=[]
for index, row in df.iterrows():
    query = row['Neighborhood']
    results = geolocator.geocode(query)
    if(results is None):
        continue
    lat = results.latitude
    long = results.longitude
    city.append(query)
    list_latitude.append(lat)
    list_longitude.append(long)

print(list_latitude)

[51.0588364, 51.1025265, 51.044845, 51.0151043, 51.0166688, 51.0446578, 51.1367865, 51.0431192, 50.8906047, 51.1405494, 51.0714621, 51.0338869, 40.7726266, 51.1285977, 37.7625125, 51.0404978, 50.9446559, 51.0892766, 50.9557843, 37.9317766, 51.0538013, 50.9007131, 51.0125403, 51.0010615, 51.120751, 51.0871956, 51.079636, 50.9428254, 38.8895563, 51.1059774, 50.9563182, 44.2302515, 50.8835939, 51.0841614, 40.7164913, 50.9831231, 51.0384338, 32.79639815, 51.1485487, 51.0344365, 51.0554215, 44.496505, 50.912048, 26.26485615, 51.0707098, 51.1430015, 51.1575003, 51.1635964, 41.7809588, 51.0596291, 51.0852147, 32.54357545, 50.9272756, 50.9202967, 50.946477, 51.0141399, -26.051166, 51.1251275, 51.0473777, 51.0464963, 51.0475537, 50.9851159, 50.9845303, 50.9616439, 52.6804443, 44.811349, 53.4702015, 51.0206343, 51.0115745, 51.0193453, 51.0290858, 42.0447388, 53.63125185, 56.0334402, 50.9915965, 51.1026919, 45.1254974, 50.6445197, 38.8103511, 51.033333, 51.0346729, 37.9765409, 51.013442, 51.02330

In [3]:
df=pd.DataFrame(city,columns=['Neighbourhood'])
df['Latitude']=list_latitude
df['Longitude']=list_longitude
df.head()

,Neighbourhood,Latitude,Longitude
0,Abbeydale,51.058836,-113.929413
1,Acadia,51.102527,-110.213738
2,Albert Park/Radisson Heights,51.044845,-113.990195
3,Altadore,51.015104,-114.100756
4,Alyth/Bonnybrook,51.016669,-114.024294


##  Using Four Square API

In [4]:
CLIENT_ID = 'WNU1KZFZTXHPHMO1XOE2L2PSQCSP34AQOI3TYZ1TM4TEABTZ' # your Foursquare ID
CLIENT_SECRET = 'ZLVJGLFOWP5YKNKHHY4FAHZJG42QXUL43KAQVROXATPS25LY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WNU1KZFZTXHPHMO1XOE2L2PSQCSP34AQOI3TYZ1TM4TEABTZ
CLIENT_SECRET:ZLVJGLFOWP5YKNKHHY4FAHZJG42QXUL43KAQVROXATPS25LY


In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
LIMIT=100
calgary_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
calgary_venues.head()

Abbeydale
Acadia
Albert Park/Radisson Heights
Altadore
Alyth/Bonnybrook
Applewood Park
Arbour Lake
Aspen Woods
Auburn Bay
Aurora Business Park
Banff Trail
Bankview
Bayview
Beddington Heights
Bel-Aire
Beltline
Bonavista Downs
Bowness
Braeside
Brentwood
Bridgeland/Riverside
Bridlewood
Britannia
Burns Industrial
Calgary International Airport
Cambrian Heights
Canada Olympic Park
Canyon Meadows
Capitol Hill
Castleridge
Cedarbrae
CFB Currie
Chaparral
Charleswood
Chinatown
Chinook Park
Christie Park
Citadel
Cityscape
Cliff Bungalow
Coach Hill
Collingwood
Copperfield
Coral Springs
Cougar Ridge
Country Hills
Country Hills Village
Coventry Hills
Cranston
Crescent Heights
Crestmont
Dalhousie
Deer Ridge
Deer Run
Diamond Cove
Discovery Ridge
Douglasdale/Glen
Dover
Downtown Commercial Core
Downtown East Village
Downtown West End
Eagle Ridge
East Fairview Industrial
East Shepard Industrial
Eastfield
Eau Claire
Edgemont
Elbow Park
Elboya
Erin Woods
Erlton
Evanston
Evergreen
Fairview
Fairview Industria

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbeydale,51.058836,-113.929413,Subway,51.059215,-113.934836,Sandwich Place
1,Abbeydale,51.058836,-113.929413,Mac's,51.059376,-113.934425,Convenience Store
2,Abbeydale,51.058836,-113.929413,roadside pub,51.059277,-113.934529,Wings Joint
3,Albert Park/Radisson Heights,51.044845,-113.990195,7-Eleven,51.047371,-113.989308,Convenience Store
4,Albert Park/Radisson Heights,51.044845,-113.990195,New Dynasty,51.047540,-113.990004,Asian Restaurant


## Extracting Restaurants

In [7]:
calgary_venues_restaurant = calgary_venues.loc[calgary_venues['Venue Category'].str.contains("Restaurant")]
calgary_venues_restaurant.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,Albert Park/Radisson Heights,51.044845,-113.990195,New Dynasty,51.047540,-113.990004,Asian Restaurant
9,Altadore,51.015104,-114.100756,Pegasus Restaurant,51.010767,-114.100119,Greek Restaurant
11,Alyth/Bonnybrook,51.016669,-114.024294,Dutch Cash & Carry,51.020801,-114.022336,Restaurant
21,Aspen Woods,51.043119,-114.210185,Edo Japan,51.041333,-114.210438,Japanese Restaurant
24,Aspen Woods,51.043119,-114.210185,Diner Deluxe Aspen,51.039636,-114.209193,Restaurant


## Using Folium for Data Visualization

In [11]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium.plugins

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [8]:
locations = calgary_venues_restaurant[['Venue Latitude', 'Venue Longitude']]
locationlist = locations.values.tolist()
len(locationlist)


469

In [12]:
map2 = folium.Map(location=[51.04, -114.21], tiles='Stamen Terrain', zoom_start=11)
marker_cluster = folium.plugins.MarkerCluster().add_to(map2)
point=0
for index,row in calgary_venues_restaurant.iterrows():
    folium.Marker(locationlist[point], popup=row['Neighbourhood'], icon=folium.Icon(color='darkblue', icon_color='white', icon='male', angle=0, prefix='fa')).add_to(marker_cluster)
    point=point+1
map2

## No.of restaurants per Neighborhood

In [13]:
calgary_venues_restaurant['Neighbourhood'].value_counts()

Manchester                      40
Chinatown                       37
Mayfair                         20
Beltline                        20
Parkdale                        18
Lower Mount Royal               18
Royal Oak                       15
Downtown Commercial Core        15
Richmond                        14
Cliff Bungalow                  13
Ogden                           13
Hillhurst                       10
Tuxedo Park                      9
Dover                            7
Lincoln Park                     7
Brentwood                        7
Crescent Heights                 6
Maple Ridge                      6
Varsity                          6
Eau Claire                       6
Country Hills Village            5
Aspen Woods                      5
Highland Park                    5
Kelvin Grove                     5
East Fairview Industrial         5
McKenzie Towne                   4
Wildwood                         4
Collingwood                      4
Mission             